In [1]:
import pandas as pd

df=pd.read_csv('./train.csv',sep=',')
df=df.fillna(0)
df.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004


In [2]:
label=df['fraud_ind']
df=df.drop(columns=['fraud_ind','txkey'])
feature=df

df_feature=pd.get_dummies(data=feature,columns=['ecfg','flbmk','flg_3dsmk','insfg','ovrlt'])
df_feature.head()

,acqic,bacno,cano,conam,contp,csmcu,etymd,hcefg,iterm,locdt,...,flbmk_0,flbmk_N,flbmk_Y,flg_3dsmk_0,flg_3dsmk_N,flg_3dsmk_Y,insfg_N,insfg_Y,ovrlt_N,ovrlt_Y
0,6881,113261,38038,513.80,5,0,0,5,0,33,...,0,1,0,0,1,0,1,0,1,0
1,0,134508,45725,465.62,5,0,2,0,0,9,...,0,1,0,0,1,0,1,0,1,0
2,6881,15408,188328,513.80,5,0,0,5,0,6,...,0,1,0,0,1,0,1,0,1,0
3,6716,157159,29967,1016.11,5,62,5,5,0,5,...,0,1,0,0,1,0,1,0,1,0
4,5975,105985,81305,713.66,5,62,4,5,0,6,...,0,1,0,0,1,0,1,0,1,0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

ndarray=df_feature.values
minmax_scale=preprocessing.MinMaxScaler(feature_range=(0,1))
scaledFeatures=minmax_scale.fit_transform(ndarray)

x_train,x_test,y_train,y_test=train_test_split(scaledFeatures,label,test_size=0.2,random_state=1000)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

model=RandomForestClassifier(n_estimators=93,max_depth=200)
model.fit(x_train,y_train)

predict1=model.predict(x_test)
predict2=model.predict(x_train)
score1=f1_score(y_test, predict1, average='binary') 
score2=f1_score(y_train, predict2, average='binary') 
print(score1,score2)

0.7670886075949367 0.9997851508547928


In [5]:
df_test=pd.read_csv('./test.csv',sep=',')
df_test=df_test.fillna('0')
df_test=df_test.drop(columns=['txkey'])
feature=df_test
df_test_feature=pd.get_dummies(data=feature,columns=['ecfg','flbmk','flg_3dsmk','insfg','ovrlt'])
test_ndarray=df_test_feature.values
test_scaledFeatures=minmax_scale.fit_transform(test_ndarray)

predict=model.predict(test_scaledFeatures)

sub=pd.read_csv('./submission_test.csv',sep=',')
sub['fraud_ind']=predict
sub.to_csv('predict.csv',index=False)